<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment01/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 133kB 46.0MB/s 
     |████████████████████████████████| 163kB 55.9MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=8c3d9dc2e4d9fe7b8e45964e1602f7b9801d6b5c8a73a03901de287d4c25b0a5
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=6900fbcb5c4c5607dd32a21e62b7132585249c3b3dc3f9ceb694ede144ef28bd
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

﻿

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
label = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(10):
   wandb.init(project="cs6910-assignment01")
   wandb.run.name = "Q1_run_{}".format(i+1)
   for j in range(30):
     if y_train[j] == i:
       wandb.log({"examples": [wandb.Image(X_train[j], caption=label[i])]})
       break

# Question 2 (10 Marks)

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [3]:
import numpy as np


def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

def tanh(z):
  g = np.tanh(z)
  return g

def relu(z):
  return np.maximum(0,z)

In [4]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [5]:
def initialize(initializer, size1, size2):
  if(initializer == "random"):
     W = np.random.randn(size1, size2) * 0.01
     return W

  if(initializer == "Xavier"):
     W = np.random.randn(size1, size2) * np.sqrt(1/size2)
     return W

  print("Enter the name of initializer correctly")

In [6]:
def linear_forward(H, W, b):
  W = np.asarray(W)
  H = np.reshape(H,(H.shape[0],-1))
  A = np.dot(W,H) + b 
  cache = (H, W, b)
  
  return A, cache

In [7]:
def initialize_Wb_matrix(X, num_hidden, size_hidden, initializer):
  layer_dims = [X.shape[0]]
  for l in range(0, num_hidden):
    layer_dims.append(size_hidden)
  layer_dims.append(10)   
  np.random.seed(3)
  Wb_matrix = {}
  update = {}
  grads = {}
  L = len(layer_dims)            # number of layers in the network

  for l in range(1,L):
    Wb_matrix['W' + str(l)] = initialize(initializer, layer_dims[l], layer_dims[l-1])
    update['W' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    Wb_matrix['b' + str(l)] = np.zeros((layer_dims[l], 1))
    update['b' + str(l)] = np.zeros((layer_dims[l], 1))
    grads['dW' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    grads['db' + str(l)] = np.zeros((layer_dims[l], 1))
  return Wb_matrix, update, grads

In [8]:
def activation_forward(H_prev, W, b, activation):
  A, linear_cache = linear_forward(H_prev, W, b)
  activation_cache = A
  if activation == 'relu':
    H = relu(A)
  elif activation == 'sigmoid':
    H = sigmoid(A)
  elif activation =='tanh':
    H = tanh(A)
  elif activation == 'softmax':
    H = softmax(A)
  
  return H, activation_cache, linear_cache

In [9]:
def forward_propagation(X, Wb_matrix, activation):
    H = X
    L = int((len(Wb_matrix)/2))
    A_caches = []
    H_caches = [H]
    for l in range(1, L):
        H_prev = H 
        H, A_cache, linear_cache = activation_forward(H_prev, Wb_matrix['W{:d}'.format(l)], Wb_matrix['b{:d}'.format(l)], activation)
        A_caches.append(A_cache)
        H_caches.append(H)
    HL, AL, linear_cache = activation_forward(H, Wb_matrix['W%d' % L], Wb_matrix['b%d' % L], activation='softmax')
    A_caches.append(AL)
    H_caches.append(HL)
    return HL, H_caches, A_caches

# Question 3 (18 Marks)

Implement the backpropagation algorithm with support for the following optimisation functions 

- sgd
- momentum based gradient descent
- nesterov accelerated gradient descent
- rmsprop
- adam
- nadam

(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)

We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.

In [10]:
def one_hot_encoding(l, L):
  import numpy
  e = []
  for i in range(L):
    if i == l:
      e.append(1)
    else:
      e.append(0)
  e_y = np.asarray(e)
  return e_y

In [11]:
def deriv_activ(z, activation):
  if(activation == "relu"):
    z[z<=0] = 0
    z[z>0] = 1
    return z

  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv
  
  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2
    return deriv

In [12]:
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation, weight_decay):
  e_y = []
  L = int((len(Wb_matrix)/2))
  num_train = y_hat.shape[1]
  num_classes = y_hat.shape[0]
  for j in range(len(y)):
    e_y.append(one_hot_encoding(y[j],num_classes))
  e_y = np.reshape(e_y,(num_classes,num_train))
  da = -(e_y - y_hat)
  grads = {}
  
  for k in reversed(range(1,L+1)):
    dW = np.matmul(da, np.transpose(H_caches[k-1])) + 2*weight_decay*Wb_matrix['W{:d}'.format(k)]
    db = da
    if k != 1:
      dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
      da = np.multiply(dh, deriv_activ(A_caches[k-2], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db
  
  return grads

In [13]:
def sgd(Wb_matrix, grads, eta,L):
  for l in range(L):
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix['b' + str(l+1)] = Wb_matrix['b' + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update,L):
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)]) + eta * grads['dW' + str(l+1)]
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]
  
  return Wb_matrix, update

def nag(Wb_matrix,eta,gamma, update, L):
  Wb_matrix_temp= {}
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)])
    Wb_matrix_temp['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)])
    Wb_matrix_temp["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix_temp, update

def rmsprop(Wb_matrix, eta, update ,beta1, eps, grads, L):
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(update['W' + str(l+1)]+eps)) * grads['dW' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(update['b' + str(l+1)]+eps)) * grads['db' + str(l+1)]
  return Wb_matrix, update
  
def adam(Wb_matrix, eta, beta1, beta2, update, v_n, eps, grads, L,i):
  
  m_hat = update.copy()
  v_hat = update.copy()
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]
    update['b' + str(l+1)] = beta1 * update['b' + str(l+1)] + (1-beta1) * grads['db' + str(l+1)]
    
    v_n['W' + str(l+1)] = beta2 * v_n['W' + str(l+1)] + (1-beta2) * grads['dW' + str(l+1)]**2
    v_n["b" + str(l+1)] = beta2 * v_n["b" + str(l+1)] + (1-beta2) * grads['db' + str(l+1)]**2
    
    m_hat['W' + str(l+1)] = update['W' + str(l+1)]/(1-math.pow(beta1,i+1))
    m_hat['b' + str(l+1)] = update['b' + str(l+1)]/(1-math.pow(beta1,i+1))

    v_hat['W' + str(l+1)] = v_n['W' + str(l+1)]/(1-math.pow(beta2,i+1))
    v_hat['b' + str(l+1)] = v_n['b' + str(l+1)]/(1-math.pow(beta2,i+1))


    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/(np.sqrt(v_hat['W' + str(l+1)]+eps)))* m_hat['W' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/(np.sqrt(v_hat['b' + str(l+1)]+eps)))* m_hat['b' + str(l+1)]

 
  return Wb_matrix, update, v_n

In [15]:
def swp():
  from keras.datasets import fashion_mnist
  from matplotlib import pyplot
  import numpy as np
  import math
  import random
  (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
  X_train_flatten = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255.0
  X_train_flatten = X_train_flatten.T
  num_train = 10000
  num_val = int(0.1*num_train)
  
  val = random.sample(range(int(num_train)), int(num_val))

  X_val = np.zeros([X_train.shape[1]*X_train.shape[2], num_val])
  y_val = np.zeros(num_val)
  for a in range(num_val):
    X_val[:,a] = X_train_flatten[:,int(val[a])]
    y_val[a] = y_train[int(val[a])]

  hyperparameter_defaults = dict(
      num_hidden = 4,
      size_hidden = 128,
      weight_decay = 0,
      optimizer = 'adam',
      initializer = 'Xavier',
      activation = 'relu',
      batch_size = 64,
      learning_rate = 0.001,
      max_epoch = 5,
      )


  wandb.init(project="cs6910-assignment01", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "hl{}_bs_{}_ac_{}".format(config.num_hidden,config.batch_size, config.activation)
  L = config.num_hidden+1
  beta1 = 0.9
  beta2 = 0.999
  eps = 1e-8
  gamma = 0.9


  Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, config.num_hidden , config.size_hidden ,config.initializer)
  v = update.copy()
  m_hat = update.copy()
  v_hat = update.copy()

  
  for i in range(config.max_epoch):
    loss = 0;
    val_loss = 0;
    count = 0;
    grads = grad_initial
    print("Epoch:", i+1)
    for j in range(num_train):
      train_ip= np.reshape(X_train_flatten[:,j], (-1, 1)) 
      y_hat, H_caches, A_caches = forward_propagation(train_ip, Wb_matrix ,config.activation)
      if(y_train[j]==np.argmax(y_hat)):
        count = count + 1; 

      if (j+1)%config.batch_size == 0  :
        if config.optimizer == 'nag':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)
        elif config.optimizer == 'nadam':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)

      grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], config.activation, config.weight_decay)
      for l in range(L):
        grads['dW' + str(l+1)] = grads['dW' + str(l+1)] + grad['dW' + str(l+1)] 
        grads['db' + str(l+1)] = grads['db' + str(l+1)] + grad['db' + str(l+1)] 


      if (j+1)%config.batch_size == 0 : 
        for l in range(L):
          grads['dW' + str(l+1)] = grads['dW' + str(l+1)]/config.batch_size
          grads['db' + str(l+1)] = grads['db' + str(l+1)]/config.batch_size
        if config.optimizer == 'sgd':
          Wb_matrix = sgd(Wb_matrix, grads, config.learning_rate,L)
        elif config.optimizer == 'momentum':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'nag':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'rmsprop':
          Wb_matrix, update = rmsprop(Wb_matrix, config.learning_rate, update,  beta1, eps, grads, L)
        elif config.optimizer == 'adam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
        elif config.optimizer == 'nadam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)    
      
      loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
    
    y_hat_val, H_caches_val, A_caches_val = forward_propagation(X_val, Wb_matrix ,config.activation)
    count_val = np.sum(np.argmax(y_hat_val, axis = 0)== y_val)
    
    for j in range(num_val):
      val_ip= np.reshape(X_val[:,j], (-1, 1)) 
      y_hat_val, H_caches_val, A_caches_val = forward_propagation(val_ip, Wb_matrix ,config.activation)
      val_loss = val_loss - (1/num_val)*math.log(y_hat_val[int(y_val[j])])

    accuracy = 100*count/num_train
    val_accuracy = 100*count_val/num_val

    print("     Loss", loss)
    print("     Accuracy",accuracy)
    print("     Validation Loss", val_loss)
    print("     Validation Accuracy", val_accuracy)

    metrics = {'epoch':i, 'val_accuracy': val_accuracy, 'val_loss': val_loss, 'accuracy': accuracy, 'loss': loss}
    wandb.log(metrics)
  wandb.run.finish()

In [16]:
import wandb

sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "cs6910-assignment01",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values": [5,10]
        },
        "num_hidden": {
            "values":[3,4,5]
        }, 
        "size_hidden": {
            "values":[32,64,128]
        },
        "weight_decay":{
            "values":[0,0.0005,0.5]
        },
        "learning_rate":{
            "values":[0.001,0.0001]
        },
        "batch_size": {
            "values":[16,32,64]
        },  
        "optimizer": {
            "values":['momentum' ,'nag','rmsprop', 'adam','nadam']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 50p0wqep
Sweep URL: https://wandb.ai/karthik21/uncategorized/sweeps/50p0wqep


In [ ]:
wandb.agent(sweep_id, swp)

wandb: Agent Starting Run: cnih3to9 with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	initializer: Xavier
wandb: 	learning_rate: 0.001
wandb: 	max_epoch: 10
wandb: 	num_hidden: 5
wandb: 	optimizer: rmsprop
wandb: 	size_hidden: 64
wandb: 	weight_decay: 0.0005


4423680/4422102 [==============================] - 0s 0us/step


wandb: Currently logged in as: karthik21 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch: 1
     Loss 0.9712336700058178
     Accuracy 62.97
     Validation Loss 0.6806922353253155
     Validation Accuracy 75.6
Epoch: 2
     Loss 0.62235560748704
     Accuracy 76.75
     Validation Loss 0.5356946377407659
     Validation Accuracy 80.3
Epoch: 3
     Loss 0.5328880562593504
     Accuracy 80.45
     Validation Loss 0.48568405351031785
     Validation Accuracy 81.7
Epoch: 4
     Loss 0.4814977027028728
     Accuracy 82.52
     Validation Loss 0.44654699774836065
     Validation Accuracy 83.8
Epoch: 5
